In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from matplotlib import style
style.use('ggplot')

Let's see the datasets

In [ ]:
df_train = pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
df_test = pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')
df_train.head()

In [ ]:
df_train.describe(include = 'all')

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

now it's clear from the above result that the dataset have no null columns

In [ ]:
print('train-set shape:', df_train.shape)
print('test-set shape:',df_test.shape)

## EDA of the Train Dataset

let's see how is the target column data counts

In [ ]:
df_train['Response'].value_counts().plot.bar()
plt.show()

Age Distribution

In [ ]:
sns.distplot(df_train['Age'])
plt.show()

Annual premium distribution

In [ ]:
sns.distplot(df_train['Annual_Premium'])
plt.show()

The distribution of annual premium is not normal, so let's take square root of the column and then  see the distribution

In [ ]:
sns.distplot(np.sqrt(df_train['Annual_Premium'].values))
plt.show()

Let's see whether people who doesn't have previously insured tend to Insure their vehicle or who is previously insured will tend to insure their vehicle

In [ ]:
plt.subplot(121)
df_train[df_train['Previously_Insured'] == 0]['Response'].value_counts().plot.bar()
plt.tight_layout()
plt.title('Previously not Insured')
plt.subplot(122)
df_train[df_train['Previously_Insured'] == 1]['Response'].value_counts().plot.bar()
plt.title('Previously Insured')
plt.tight_layout()
plt.show()

Let's see whether people whose vehicle is damage more tend to insure their vehicle or the one whose vehicle is not damage

In [ ]:
plt.subplot(121)
df_train[df_train['Vehicle_Damage'] == 'Yes']['Response'].value_counts().plot.bar()
plt.tight_layout()
plt.title('Vehicle Damage')
plt.subplot(122)
df_train[df_train['Vehicle_Damage'] == 'No']['Response'].value_counts().plot.bar()
plt.title('Vehicle not Damage')
plt.tight_layout()
plt.show()

From above two graph it is clear that peoplr whose vehicle is damage and does not have Insurance are more tend to Insured their vehicle

Now let's see is there any corrlation between  Age and Annual_Premium column

In [ ]:
sns.lmplot( x = 'Annual_Premium' , y = 'Age' , data = df_train)
plt.title('Annual Premium V/s Age')
plt.show()

So, when age of the person increases the Annual premium also increases

checking the correlation between region code and Annual premium column

In [ ]:
sns.lmplot( x = 'Annual_Premium' , y = 'Region_Code' , data = df_train)
plt.title('Annual premium V/s Region Code')
plt.show()

Region code less than 30.0 has more annual premium 

Checking the correlation between Policy_sales_channel and annual premium

In [ ]:
sns.lmplot( x = 'Annual_Premium' , y = 'Policy_Sales_Channel' ,  data =  df_train)
plt.title('Annual premium V/s Policy sales channel')
plt.show()

Policy_Sales_Channel and Annual_Premium have negative correlation 

let's chek the relation between Vintage and Annual_Premium

In [ ]:
sns.lmplot( x ='Annual_Premium' ,  y = 'Vintage' ,  data = df_train)
plt.title('Annual_Premium V/s Vintage ')
plt.show()

## Feature Engineering

In [ ]:
# let's put together the training dataset and the test dataset for that first we need to assign somthing in the
# response column of the test dataset so that we can filter test dataset once the feature engineering id done

df_test['Response'] = -1 
df = pd.concat([df_train , df_test])
print(df.shape)
df.head()

In [ ]:
# Since 'Age' and 'Annual_Premium' is correlated so let's extract a feature using these two column
df['Age_Wise_Premium'] = df['Annual_Premium'] / df['Age']

# The person whose vehicle is Damage and not Insured, more tend to Insured their vehicle So lat's make a column
# using these two column

df['Not_Insured_Damage'] = df['Previously_Insured'].astype('str')+'_'+df['Vehicle_Damage'].astype('str')

In [ ]:
df.head()

In [ ]:
# Annual_Premium region wise mean
df['Region_Wise_Mean_Premium'] = df.groupby('Region_Code')['Annual_Premium'].transform('mean')
# Annual Premium mean according to policy sales channel
df['Channel_Wise_Mean_Premium'] = df.groupby('Policy_Sales_Channel')['Annual_Premium'].transform('mean')
# Annual Premium mean baseed on vehicle age
df['Age_Wise_Mean_Premium'] = df.groupby('Vehicle_Age')['Annual_Premium'].transform('mean')

In [ ]:
df.head()

Let's see now the correlation matrix of the Numerical columns

In [ ]:
corr = df.corr()

In [ ]:
corr.style.background_gradient(cmap='coolwarm', axis = None).set_precision(2)

categorical column feature engineering